# word2vec 단어 간 유사도 체크
- 최대한 육아를 주제로 한 말뭉치를 모아보았다. (Total 2,755 documents)
- 우리가 원하는 의도에 일치하는 단어 간 유사도 결과가 나오는지를 확인하는 것이 목적이다.

## Import

In [43]:
import pandas as pd
import numpy as np

from konlpy.tag import Mecab
from gensim.models import Word2Vec

### Datas

In [44]:
df1 = pd.read_csv("../crawling/text/Maeili/maeili.csv")
df2 = pd.read_csv("../crawling/text/Momtok/MomtokCrawler/baby.csv")
df3 = pd.read_csv("../crawling/text/Momtok/MomtokCrawler/toys.csv")

seoulchildcare = []

with open("../crawling/text/SeoulChildcare/seoulchildcare.txt", "r") as textfile:
    for row in textfile:
        seoulchildcare.append(row)
        
df4 = pd.DataFrame({'text': seoulchildcare})

#### maeili(매일아이)

In [45]:
df1.tail()

,categories,title,text,postlink
1533,임신,'웃음요법' 태교,임신 중에는 스트레스를 받지 않고 몸을 편안하게 하는 것만큼 좋은 태교가 없습니다 ...,https://www.maeili.com/cms/contents/contentsVi...
1534,임신,기형 검사와 보조 검사,모든 임신부는 나이에 상관없이 7백 명에 한 명꼴로 정신박약을 초래하는 다운 증후군...,https://www.maeili.com/cms/contents/contentsVi...
1535,임신,임신 시기별 검사 종류와 횟수,첫방문시 혈액검사 소변검사 혈액형 검사 풍진항체 검사 8 18주 초음파검사 필요한...,https://www.maeili.com/cms/contents/contentsVi...
1536,임신,초음파 검사의 임상적 유용성,초기의 정상적인 자궁내 임신의 확인 임신낭과 태아의 성장 여부와 크기 측정 임신주...,https://www.maeili.com/cms/contents/contentsVi...
1537,임신,산전 유전 상담이란,유전 질환 염색체 이상도 말초 혈액의 염색체 검사로서 진단이 가능하나 산과 영역에서...,https://www.maeili.com/cms/contents/contentsVi...


#### momtalk(육아정보)

In [46]:
df2.tail()

,idx,text
695,852,유아용품 유통기한 바로알기 기저귀 화장품 기저귀 화장품에도 유통기한이 있다구 두고...
696,928,우리 아이 쑥쑥 클 수 있게 전지현 크림으로 하는 베이비 마사지법 서 알려드려요 ...
697,971,이유식을 시작하는 맘들을 위해 이유식 준비물 요점 정리 이유식을 시작하는 맘들을 ...
698,843,휴일에 문 여는 약국이 궁금하다면 휴일지킴이약국 검색법 하필 쉬는 날 응급약이 필...
699,842,우리 아이가 아파요 응급실에 가야할까요 소아과 전문의 김연호 선생님과 함께하는 우...


#### momtalk(유아용품)

In [47]:
df3.tail()

,idx,text
482,622,우리아이 치발기 치아발육기 선택 꿀팁 잇몸이 근질근질우리아이 치아발육기 선택팁 아...
483,640,쏘서 점퍼루 보행기 우리아이 선택은 쏘서 점퍼루 보행기 우리아이 선택은 아이를 낳...
484,607,신생아부터 유아까지 아기목욕용품 및 아기 목욕시키기 신생아부터 유아까지아기 목욕용...
485,567,아기용품 어떤걸로 골라야 할까 아기용품 고르기 오늘은 아기용품고르기에 대해 함께 ...
486,604,장난감수납장 이거좋아요 이번주 베스트글 주인공은 소영님 입니다 요즘 장난감수납장에...


#### 서울육아종합지원센터 육아전문칼럼

In [48]:
df4.tail()

,text
34,주제 영유아의 선행학습 진짜 도움될까 부제 당신의 잘못이 아닙니다 다만 저는 예전...
35,좋은 아빠 지금 하지 않으면 늦다 결혼을 하고 아이를 키우면서 가장 힘든 점은 욕...
36,놀이의 뇌과학적 가치 요즘 아이들은 밖에서 같이 놀 기회가 적어졌다 누군가와 놀아...
37,영유아기 자녀와 안정 애착 형성하기 영유아기 때 부모와 맺은 관계 경험은 평생을 ...
38,아이를 숲으로 데려갈까요 4차 산업혁명시대를 맞이하여 더 많이 놀게 하고 더 많이...


### Concatenate Dataframe

In [192]:
df = pd.concat([df1.text, df2.text, df3.text, df4.text])
df = pd.DataFrame(df).reset_index(drop=True)

### Remove some contents

In [193]:
df = df.drop(index=[952, 955, 956, 960, 962, 964, 1165, 1240, 1322])

### Add tokenized column

In [194]:
def tokenize(text):
    tokenizer = Mecab()
    text = tokenizer.morphs(text)
#     text = ' '.join(text)
    return text
    
df['tokenized'] = df['text'].apply(tokenize)
df = df.dropna()

In [209]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2755 entries, 0 to 2763
Data columns (total 2 columns):
text         2755 non-null object
tokenized    2755 non-null object
dtypes: object(2)
memory usage: 64.6+ KB


## Train Word2Vec

#### WTF error
- df를 csv 형식으로 저장한 후 다시 로드하면 Word2Vec 학습에 인코딩 문제가 생긴다.
- 더 자세히 파고들어보니 list 형식의 column이 csv로 저장할 때 강제로 string 타입으로 형변환 됨.
- 이렇게 형변환 된 column을 Word2Vec 학습에 사용할 시 한글 텍스트가 깨지는 현상이 발생함.. 정확한 원인은 아직 모르겠다.

In [188]:
# df.to_csv('text_for_word2vec.csv', index=False, encoding='utf-8')
# df = pd.read_csv('text_for_word2vec.csv', encoding='utf-8')

그러므로 저장 & 로드 과정 생략하고 바로 말뭉치를 numpy ndarray형식으로 변환

In [199]:
corpus = np.array(df.tokenized)
type(corpus)

numpy.ndarray

In [197]:
model = Word2Vec(corpus, size=100, workers=4, sg=1)
model.save('tokmom_word2vec')

#### 유사어 분석

In [198]:
model.wv.similar_by_word("놀이")

[('미술', 0.6401194334030151),
 ('소꿉', 0.640009343624115),
 ('악기', 0.639319658279419),
 ('가상', 0.6161954998970032),
 ('점토', 0.6113858222961426),
 ('보드게임', 0.6093951463699341),
 ('공놀이', 0.6089156866073608),
 ('블럭', 0.6015890836715698),
 ('퍼즐', 0.597853422164917),
 ('공예', 0.5955957770347595)]

In [201]:
model.wv.similar_by_word("아이")

[('아기', 0.6443846225738525),
 ('외동아이', 0.638481855392456),
 ('궁금해하', 0.635124683380127),
 ('말다툼', 0.6187405586242676),
 ('가령', 0.61295485496521),
 ('아가', 0.6117354035377502),
 ('큰아이', 0.6047983169555664),
 ('참을성', 0.5999466180801392),
 ('자녀', 0.5994678735733032),
 ('언제나', 0.5962915420532227)]

In [202]:
model.wv.similar_by_word("치아")

[('유치', 0.7831703424453735),
 ('영구치', 0.7581613659858704),
 ('충치', 0.6993371248245239),
 ('잇몸', 0.6968017816543579),
 ('젖니', 0.6424744725227356),
 ('발육기', 0.6411329507827759),
 ('우식증', 0.636883020401001),
 ('턱뼈', 0.6170935034751892),
 ('관리', 0.6111301183700562),
 ('치과', 0.5850251913070679)]

In [203]:
model.wv.similar_by_word("이빨")

[('다물', 0.8562512397766113),
 ('물렸', 0.855617344379425),
 ('묻히', 0.8498890995979309),
 ('흘린', 0.8470172882080078),
 ('눕혀서', 0.8447372317314148),
 ('갈아입혀', 0.8428230285644531),
 ('빨아먹', 0.8425527811050415),
 ('비우', 0.841046929359436),
 ('문질러서', 0.8384517431259155),
 ('뗀', 0.8335757255554199)]

In [204]:
model.wv.similar_by_word("감기")

[('기관지염', 0.8151572942733765),
 ('구내염', 0.8048379421234131),
 ('코감기', 0.7859163284301758),
 ('모세', 0.7803899645805359),
 ('냉방병', 0.7658597230911255),
 ('독감', 0.7635818719863892),
 ('족구', 0.7625694274902344),
 ('일교차', 0.7553218603134155),
 ('전염병', 0.7465672492980957),
 ('비염', 0.746048092842102)]

In [205]:
model.wv.similar_by_word("열")

[('고열', 0.6815513372421265),
 ('오르', 0.6455028057098389),
 ('떨어졌', 0.6262140274047852),
 ('오한', 0.6194145679473877),
 ('배탈', 0.6173418164253235),
 ('진물', 0.6168859601020813),
 ('38', 0.6107426881790161),
 ('실밥', 0.6009123921394348),
 ('콧물', 0.5945661067962646),
 ('맥박', 0.591853141784668)]

In [206]:
model.wv.similar_by_word("아파")

[('운다', 0.8518242835998535),
 ('가려워하', 0.8357567191123962),
 ('신음', 0.8261104822158813),
 ('숨소리', 0.8246290683746338),
 ('아파서', 0.8228338956832886),
 ('쌕쌕거리', 0.8164627552032471),
 ('아파하', 0.8094748258590698),
 ('아프', 0.8033629059791565),
 ('졸리', 0.8015425205230713),
 ('자지러지', 0.7976456880569458)]

In [208]:
model.wv.similar_by_word("아프")

[('고프', 0.8295062780380249),
 ('아파서', 0.8044006824493408),
 ('아파', 0.8033629059791565),
 ('아픈', 0.8004787564277649),
 ('시리', 0.7910274267196655),
 ('신음', 0.7867963314056396),
 ('가렵', 0.7781084775924683),
 ('보채', 0.7769567966461182),
 ('몹시', 0.7743347883224487),
 ('아파하', 0.7673138380050659)]

In [207]:
model.wv.similar_by_word("울음")

[('지르', 0.7875815629959106),
 ('터뜨리', 0.7765035629272461),
 ('비명', 0.7734931111335754),
 ('투정', 0.7645355463027954),
 ('웃음', 0.7563694715499878),
 ('짜증', 0.7504301071166992),
 ('좌절', 0.748199462890625),
 ('떼쓰', 0.743984580039978),
 ('눈치', 0.7426326274871826),
 ('놀라', 0.7379218339920044)]

---

## Only Noun

- 이번에는 말뭉치에서 명사만 추출하여 결과를 비교해보자

In [217]:
def extract_noun(text):
    tokenizer = Mecab()
    text = tokenizer.nouns(text)
#     text = ' '.join(text)
    return text
    
df['noun'] = df['text'].apply(extract_noun)
df = df.dropna()

In [219]:
noun_corpus = np.array(df.noun)

In [220]:
model = Word2Vec(noun_corpus, size=100, workers=4, sg=1)
# model.save('tokmom_word2vec_only_noun')

In [221]:
model.wv.similar_by_word("놀이")

[('소꿉', 0.7232999801635742),
 ('가상', 0.6864834427833557),
 ('퍼즐', 0.6853044033050537),
 ('공구', 0.6801934242248535),
 ('블럭', 0.6749951839447021),
 ('놀잇감', 0.6738475561141968),
 ('모래', 0.6715108156204224),
 ('숨바꼭질', 0.6682935357093811),
 ('교구', 0.6666603088378906),
 ('낚시', 0.6649571657180786)]

In [222]:
model.wv.similar_by_word("아이")

[('예전', 0.7032129168510437),
 ('걸', 0.6790162324905396),
 ('여든', 0.6752416491508484),
 ('넌', 0.6690855026245117),
 ('실랑이', 0.6682751178741455),
 ('요령', 0.6678499579429626),
 ('의문', 0.6665750741958618),
 ('만약', 0.6650289297103882),
 ('예고', 0.6623905897140503),
 ('거기', 0.6621741056442261)]

In [223]:
model.wv.similar_by_word("치아")

[('유치', 0.887710690498352),
 ('영구치', 0.8551468849182129),
 ('충치', 0.8333171606063843),
 ('잇몸', 0.8128228187561035),
 ('젖니', 0.7964135408401489),
 ('어금니', 0.7822399139404297),
 ('발육기', 0.7612699270248413),
 ('치아우식증', 0.7477948665618896),
 ('법랑질', 0.728696346282959),
 ('불소', 0.7240395545959473)]

In [224]:
model.wv.similar_by_word("이빨")

[('젖떼기', 0.8597941398620605),
 ('치아우식증', 0.8539888262748718),
 ('결심', 0.8492799401283264),
 ('양치질', 0.8481295704841614),
 ('허기', 0.8437939286231995),
 ('다면', 0.8409777283668518),
 ('안팎', 0.8391331434249878),
 ('아파', 0.8380986452102661),
 ('쉬야', 0.8365988731384277),
 ('생략', 0.8350856304168701)]

In [225]:
model.wv.similar_by_word("감기")

[('기관지염', 0.8361650109291077),
 ('구내염', 0.8307960629463196),
 ('일교차', 0.8283977508544922),
 ('독감', 0.8273149132728577),
 ('발병', 0.8210816383361816),
 ('냉방병', 0.8203357458114624),
 ('환절기', 0.8194378018379211),
 ('모세', 0.8119893074035645),
 ('호흡기', 0.8116012811660767),
 ('고열', 0.8019291162490845)]

In [226]:
model.wv.similar_by_word("열")

[('고열', 0.7782238721847534),
 ('해열제', 0.7610315084457397),
 ('고열로', 0.758381724357605),
 ('경련', 0.7324706315994263),
 ('오한', 0.7209625244140625),
 ('미열', 0.7193896770477295),
 ('감기', 0.7161926031112671),
 ('기운', 0.7157675623893738),
 ('성경', 0.7133293151855469),
 ('발열', 0.711622953414917)]

In [227]:
model.wv.similar_by_word("아파")

[('자다', 0.9248478412628174),
 ('안팎', 0.9233746528625488),
 ('아랫입술', 0.9227613210678101),
 ('와중', 0.9191538095474243),
 ('부근', 0.9186658263206482),
 ('다거', 0.9164068698883057),
 ('간헐', 0.9160115122795105),
 ('사레', 0.9143134951591492),
 ('지름', 0.9142541289329529),
 ('저부', 0.9125714302062988)]

In [229]:
model.wv.similar_by_word("울음")

[('비명', 0.8273400068283081),
 ('눈빛', 0.8135977983474731),
 ('미소', 0.8038680553436279),
 ('옹알이', 0.8020099401473999),
 ('부리기', 0.7986065149307251),
 ('불만', 0.7965575456619263),
 ('젖먹이', 0.7963262796401978),
 ('대꾸', 0.7940142750740051),
 ('내게', 0.7907352447509766),
 ('어조', 0.7901761531829834)]

### Volume of Corpus 

In [233]:
df[['text']].to_csv('text.csv', index=False, encoding='utf-8')

In [235]:
pd.read_csv('text.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2754 entries, 0 to 2753
Data columns (total 1 columns):
text    2754 non-null object
dtypes: object(1)
memory usage: 21.6+ KB
